In [48]:
import pydantic
print(pydantic.__version__)

2.10.6


## Defining schemas in pydantic

Create a model for a user with
* a username
* an email address
* an ID (keep it as an integer for now)
* a date of birth

Without pydantic (and any library), it would look like this

In [49]:
from datetime import datetime

class User:
    def __init__(self, id, username, email, dob):
        # Validate id (must be an int)
        # if i want `id` to be an integer, i check if it is an instance of the int type
        # and if not i raise a error
        if not isinstance(id, int):
            raise TypeError(f'Expected id to be an int, got {type(id).__name__}')
        
        # Validate username (must be a string)
        if not isinstance(username, str):
            raise TypeError(f'Expected username to be a str, got {type(username).__name__}')
        
        # Validate email (must be a string and basic email format check)
        if not isinstance(email, str):
            raise TypeError(f'Expected email to be a str, got {type(email).__name__}')
        if '@' not in email or '.' not in email.split('@')[-1]:
            raise ValueError(f'Invalid email format: {email}')
        
        # Validate dob (must be a datetime object)
        if not isinstance(dob, datetime):
            raise TypeError(f'Expected dob to be a datetime, got {type(dob).__name__}')
        
        # Assign values to instance variables
        self.id = id
        self.username = username
        self.email = email
        self.dob = dob

In [50]:
# Example usage
try:
    user = User(id=123, username='john_doe', email='john.doe@example.com', dob=datetime(1990, 5, 10))
    print("User created successfully:", user.__dict__)
except (TypeError, ValueError) as e:
    print(f"Error: {e}")

# Example with invalid email
try:
    invalid_user = User(id=123, username='john_doe', email='john.doe@example', dob=datetime(1990, 5, 10))
except (TypeError, ValueError) as e:
    print(f"Error: {e}")

User created successfully: {'id': 123, 'username': 'john_doe', 'email': 'john.doe@example.com', 'dob': datetime.datetime(1990, 5, 10, 0, 0)}
Error: Invalid email format: john.doe@example


That did the following validations
* id validation: The id must be an integer (int). If it's not, a TypeError is raised.
* username validation: The username must be a string (str).
* email validation: The email must be a string, and we perform a basic validation to check that it contains both @ and . to simulate a basic email format check.
* dob validation: The dob must be a datetime object.

`dataclasses` can somewhat simplify the syntax but still not ideal

In [51]:
from dataclasses import dataclass, field

@dataclass
class User:
    id: int
    username: str
    email: str
    dob: datetime

    def __post_init__(self):
        # Validate id (must be an int)
        if not isinstance(self.id, int):
            raise TypeError(f'Expected id to be an int, got {type(self.id).__name__}')
        
        # Validate username (must be a string)
        if not isinstance(self.username, str):
            raise TypeError(f'Expected username to be a str, got {type(self.username).__name__}')
        
        # Validate email (must be a string and basic email format check)
        if not isinstance(self.email, str):
            raise TypeError(f'Expected email to be a str, got {type(self.email).__name__}')
        if '@' not in self.email or '.' not in self.email.split('@')[-1]:
            raise ValueError(f'Invalid email format: {self.email}')
        
        # Validate dob (must be a datetime object)
        if not isinstance(self.dob, datetime):
            raise TypeError(f'Expected dob to be a datetime, got {type(self.dob).__name__}')

In [52]:
# Example usage
try:
    user = User(id=123, username='john_doe', email='john.doe@example.com', dob=datetime(1990, 5, 10))
    print("User created successfully:", user)
except (TypeError, ValueError) as e:
    print(f"Error: {e}")

# Example with invalid email
try:
    invalid_user = User(id=123, username='john_doe', email='john.doe@example', dob=datetime(1990, 5, 10))
except (TypeError, ValueError) as e:
    print(f"Error: {e}")

User created successfully: User(id=123, username='john_doe', email='john.doe@example.com', dob=datetime.datetime(1990, 5, 10, 0, 0))
Error: Invalid email format: john.doe@example


With Pydantic, it looks like this

In [53]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    username: str
    email: str
    dob: datetime

# basically, extend the BaseModel class and use type annotations to define the expected data types

In [54]:
# Example usage
u = User(id=1, username="freethrow", email="email@gmail.com", dob=datetime(1975, 5, 13))

if you create a user with the wrong data, it will throw an error

In [55]:
if False:
    # Example with invalid id
    u = User(
        id="one",
        username="freethrow",
        email="email@gmail.com",
        dob=datetime(1975, 5, 13),
    )

catch the error

In [56]:
try:
    u = User(
        id="one",
        username="freethrow",
        email="email@gmail.com",
        dob=datetime(1975, 5, 13),
    )
    print(u)
except Exception as e:
    print(e)

1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='one', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


To get more error details

In [57]:
from pydantic import ValidationError

try:
    u = User(
        id="one",
        username="freethrow",
        email="email@gmail.com",
        dob=datetime(1975, 5, 13),
    )
    print(u)
except ValidationError as e:
    print(e.errors())

[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'one', 'url': 'https://errors.pydantic.dev/2.10/v/int_parsing'}]


When we write

```python
class User(BaseModel):
    id: int
    username: str
    email: str
    dob: datetime
```

we are defining schema in Pydantic by creating a Model. This is one of the primary ways of defining schema in Pydantic. Models are simply classes which inherit from `BaseModel` and define fields as annotated attributes.

User is a model with two fields:

* id, which is an integer and is required
* name, which is a string and is not required (it has a default value).

The model can then be instantiated:

```python
user = User(id='123')
```

user is an instance of User. Initialization of the object will perform all parsing and validation. If no ValidationError exception is raised, you know the resulting model instance is valid.

### validating data

Pydantic provides three methods on models classes for parsing data:

* `model_validate()`: this is very similar to the __init__ method of the model, except it takes a dictionary or an object rather than keyword arguments. If the object passed cannot be validated, or if it's not a dictionary or instance of the model in question, a ValidationError will be raised.
* `model_validate_json()`: this validates the provided data as a JSON string or bytes object. If your incoming data is a JSON payload, this is generally considered faster (instead of manually parsing the data as a dictionary). Learn more about JSON parsing in the JSON section of the docs.
* `model_validate_strings()`: this takes a dictionary (can be nested) with string keys and values and validates the data in JSON mode so that said strings can be coerced into the correct types.

In [58]:
class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime|None = None

In [59]:
m = User.model_validate({'id': 123, 'name': 'James'})

m

User(id=123, name='James', signup_ts=None)

In [60]:
try:
    User.model_validate(['not', 'a', 'dict'])
except ValidationError as e:
    print(e)

1 validation error for User
  Input should be a valid dictionary or instance of User [type=model_type, input_value=['not', 'a', 'dict'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type


In [61]:
m = User.model_validate_json('{"id": 123, "name": "James"}')

m

User(id=123, name='James', signup_ts=None)

In [62]:
try:
    m = User.model_validate_json('{"id": 123, "name": 123}')
except ValidationError as e:
    print(e)

1 validation error for User
name
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


In [63]:
m = User.model_validate_strings({'id': '123', 'name': 'James'})

m

User(id=123, name='James', signup_ts=None)

In [64]:
m = User.model_validate_strings(
    {'id': '123', 'name': 'James', 'signup_ts': '2024-04-01T12:00:00'}
)

m

User(id=123, name='James', signup_ts=datetime.datetime(2024, 4, 1, 12, 0))

In [65]:
try:
    m = User.model_validate_strings(
        {'id': '123', 'name': 'James', 'signup_ts': '2024-04-01'}, strict=True
    )
except ValidationError as e:
    print(e)

1 validation error for User
signup_ts
  Input should be a valid datetime, invalid datetime separator, expected `T`, `t`, `_` or space [type=datetime_parsing, input_value='2024-04-01', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/datetime_parsing


### Required vs Optional/Nullable fields

In [66]:
class User(BaseModel):
    id: int
    name: str = 'Jane Doe' # by givin it a default, you make it optional

Required Fields:
* The field id is required because there is no default value assigned to it. In Pydantic, any field without a default value is considered required.

Optional (nullable) Fields:
* The field name has a default value 'Jane Doe', so it’s optional. If no value is provided for name, the default will be used.

In [67]:
# you can say it can be null without proving the default value explicitly
class User(BaseModel):
    id: int
    name: str | None = None

In [68]:
user = User(id=123)

user

User(id=123, name=None)

In [69]:
user.id

123

In [70]:
print(user.model_fields_set)
user = User(id=123, name="Jane Doe")
print(user.model_fields_set)

{'id'}
{'id', 'name'}


In [71]:
class User(BaseModel):
    id: int
    username: str
    email: str
    dob: datetime | None = None
    fav_colors: list[str] | None = ["red", "blue"]

In [72]:
try:
    u = User(
        id=2,
        username="freethrow",
        email="email@gmail.com",
        dob=datetime(1975, 5, 13),
    )
    print(u)
except Exception as e:
    print(e)

id=2 username='freethrow' email='email@gmail.com' dob=datetime.datetime(1975, 5, 13, 0, 0) fav_colors=['red', 'blue']


In [73]:
from pydantic import PositiveInt

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]

In [74]:
external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

In [75]:
user

User(id=123, name='John Doe', signup_ts=datetime.datetime(2019, 6, 1, 12, 22), tastes={'wine': 9, 'cheese': 7, 'cabbage': 1})

In [76]:
print(user.id)
print(user.model_dump())

123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


In [77]:
external_data = {'id': 'not an int', 'tastes': {}}

try:
    User(**external_data)
except ValidationError as e:
    print(e.errors())

[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not an int', 'url': 'https://errors.pydantic.dev/2.10/v/int_parsing'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 'not an int', 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}]


## Serialization

Beyond accessing model attributes directly via their field names (e.g. model.foobar), models can be converted, dumped, serialized, and exported in a number of ways. Pydantic uses the terms "serialize" and "dump" interchangeably. Both refer to the process of converting a model to a dictionary or JSON-encoded string.

Pydantic provides functionality to serialize model in three ways:

* To a Python dict made up of the associated Python objects.
* To a Python dict made up only of "jsonable" types.
* To a JSON

In all three modes, the output can be customized by excluding specific fields, excluding unset fields, excluding default values, and excluding None values.

https://docs.pydantic.dev/latest/concepts/serialization/

In [78]:
class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

In [79]:
print(user.model_dump()) # convert to a dict
print(user.model_dump(mode='json')) # convert to a dict made up only of "jsonable" types
print(user.model_dump_json()) # convert to json
print(user.model_json_schema())

{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}
{'id': 123, 'name': 'John Doe', 'signup_ts': '2019-06-01T12:22:00', 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}
{"id":123,"name":"John Doe","signup_ts":"2019-06-01T12:22:00","tastes":{"wine":9,"cheese":7,"cabbage":1}}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'John Doe', 'title': 'Name', 'type': 'string'}, 'signup_ts': {'anyOf': [{'format': 'date-time', 'type': 'string'}, {'type': 'null'}], 'title': 'Signup Ts'}, 'tastes': {'additionalProperties': {'exclusiveMinimum': 0, 'type': 'integer'}, 'title': 'Tastes', 'type': 'object'}}, 'required': ['id', 'signup_ts', 'tastes'], 'title': 'User', 'type': 'object'}


In [80]:
class Meeting(BaseModel):
    when: datetime
    where: bytes
    why: str = 'No idea'

In [81]:
m = Meeting(when='2020-01-01T12:00', where='home')

print(m.model_dump(exclude_unset=True))
#> {'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}

print(m.model_dump(exclude={'where'}, mode='json'))
#> {'when': '2020-01-01T12:00:00', 'why': 'No idea'}

print(m.model_dump_json(exclude_defaults=True))
#> {"when":"2020-01-01T12:00:00","where":"home"}

{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
{'when': '2020-01-01T12:00:00', 'why': 'No idea'}
{"when":"2020-01-01T12:00:00","where":"home"}


## Nested models

In [82]:
from typing import List, Optional
from pydantic import BaseModel

class Food(BaseModel):
    name: str
    price: float
    # ingredients: Optional[List[str]] = None
    ingredients: List[str] | None = None

class Restaurant(BaseModel):
    name: str
    location: str
    foods: List[Food]

In [83]:
restaurant_instance = Restaurant(
    name="Tasty Bites",
    location="123, Flavor Street",
    foods=[
        {"name": "Cheese Pizza", "price": 12.50, "ingredients": ["Cheese", "Tomato Sauce", "Dough"]},
        {"name": "Veggie Burger", "price": 8.99}
    ]
)

print(restaurant_instance)
print(restaurant_instance.model_dump())

name='Tasty Bites' location='123, Flavor Street' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Dough']), Food(name='Veggie Burger', price=8.99, ingredients=None)]
{'name': 'Tasty Bites', 'location': '123, Flavor Street', 'foods': [{'name': 'Cheese Pizza', 'price': 12.5, 'ingredients': ['Cheese', 'Tomato Sauce', 'Dough']}, {'name': 'Veggie Burger', 'price': 8.99, 'ingredients': None}]}


## Validation on pydantic fields

* email validation

```shell
pipenv install email-validator
# or
pipenc install pydantic[email]
```

In [84]:
from pydantic import EmailStr

class Model(BaseModel):
    email: EmailStr

In [85]:
print(Model(email='contact@mail.com'))

email='contact@mail.com'


* various other types

In [86]:
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr
    
class Owner(BaseModel):
    name: str
    email: EmailStr
    
class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    # Must match the regex pattern ^[a-zA-Z0-9-' ]+$.

    owner: Owner
    # Must be a dictionary with keys name (a string) and email (valid EmailStr)

    address: Address
    # Must be a dictionary with keys street, city, state, and zip_code, all as strings.

    employees: conlist(Employee, min_length=2)
    # Must be a list of Employee objects, with at least two items (min_length=2).

    number_of_seats: PositiveInt
    # Must be a positive integer

    delivery: bool

    website: HttpUrl
    # Must be a valid HttpUrl

In [87]:
# Creating an instance of the Restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name": "John Doe",
        "email": "john.doe@example.com"
    },
    address={
        "street": "123, Flavor Street",
        "city": "Tastytown",
        "state": "TS",
        "zip_code": "12345",
    },
    employees=[
        {
            "name": "Jane Doe",
            "position": "Chef",
            "email": "jane.doe@example.com"
        },
        {
            "name": "Mike Roe",
            "position": "Waiter",
            "email": "mike.roe@example.com"
        }
    ],
    number_of_seats=50,
    delivery=True,
    website="http://tastybites.com"
)

# Printing the instance
print(restaurant_instance)

name='Tasty Bites' owner=Owner(name='John Doe', email='john.doe@example.com') address=Address(street='123, Flavor Street', city='Tastytown', state='TS', zip_code='12345') employees=[Employee(name='Jane Doe', position='Chef', email='jane.doe@example.com'), Employee(name='Mike Roe', position='Waiter', email='mike.roe@example.com')] number_of_seats=50 delivery=True website=HttpUrl('http://tastybites.com/')


The `Field` class is used to customize models and add metadata to the model fields.

https://docs.pydantic.dev/latest/api/fields/

In [88]:
from pydantic import  Field

# eg. model a product
class Product(BaseModel):
    product_id: int = Field(..., alias="id", gt=0, description="Unique identifier for the product")
    product_name: str = Field(..., alias="name", max_length=50, description="Name of the product")
    price: float = Field(..., alias="productPrice", ge=0.0, description="Price of the product")
    in_stock: bool = Field(..., alias="isAvailable", description="Availability status of the product")

In [89]:
# Creating an instance using aliases
data = {
    "id": 101,
    "name": "Laptop",
    "productPrice": 999.99,
    "isAvailable": True,
}

product = Product(**data)

In [90]:
# Accessing attributes with original field names
print(product.product_id)
print(product.product_name)

# Accessing the original dict with aliases
print(product.model_dump(by_alias=True))

101
Laptop
{'id': 101, 'name': 'Laptop', 'productPrice': 999.99, 'isAvailable': True}


In [91]:
# eg. model a chess event

from uuid import uuid4

class ChessTournament(BaseModel):
    id: int = Field(strict=True) # Ensures the value must strictly be an integer, rejecting float values (e.g., 5.0)
    dt: datetime = Field(default_factory=datetime.now) # Automatically assigns the current datetime when the model instance is created, unless a value for dt is explicitly provided
    name: str = Field(min_length=10, max_length=30)
    num_players: int = Field(ge=4, le=16, multiple_of=2)
    code: str = Field(default_factory=uuid4) # Generates a unique identifier to each tournament

in `dt: datetime = Field(default_factory=datetime.now)`, `default_factory` executes the function (datetime.now) each time a new model instance is created.

in `dt: datetime = datetime.now()`, `datetime.now()` function is evaluated once at the time the code is loaded, i.e., when the Python interpreter executes the class definition. All instances of the model share this same value as their default.

In [92]:
class UserModel(BaseModel):
    id: int = Field()
    username: str = Field(min_length=5, max_length=20)
    email: EmailStr = Field()
    password: str = Field(min_length=5, max_length=20, pattern="^[a-zA-Z0-9]+$") # only alphanumeric characters and no spaces

In [93]:
u = UserModel(
    id=1,
    username="freethrow",
    email="email@gmail.com",
    password="password123",
)

print(u.model_dump())

{'id': 1, 'username': 'freethrow', 'email': 'email@gmail.com', 'password': 'password123'}


In [94]:
# dump the model to a JSON representation and omit the password for security reasons
print(u.model_dump_json(exclude=set("password")))

{"id":1,"username":"freethrow","email":"email@gmail.com","password":"password123"}


## Custom serializers and custom field validators

In [95]:
from pydantic import BaseModel, Field, EmailStr, field_validator, model_serializer
from datetime import datetime

class User(BaseModel):
    id: int
    username: str = Field(min_length=5, max_length=20)
    email: EmailStr
    date_joined: datetime = Field(default_factory=datetime.now)
    is_active: bool = True

    # Custom validator for username
    @field_validator("username")
    def validate_username(cls, value):
        if not value.isalnum():
            raise ValueError("Username must be alphanumeric.")
        if value.lower() == "admin":
            raise ValueError("Username 'admin' is not allowed.")
        return value

    # Custom validator for email
    @field_validator("email")
    def validate_email_domain(cls, value):
        allowed_domains = {"example.com", "test.com"}
        if value.split("@")[-1] not in allowed_domains:
            raise ValueError("Email domain must be one of: example.com, test.com.")
        return value

    # Custom serializer to modify how the object is represented in JSON
    @model_serializer
    def serialize_model(self) -> dict:
        data = self.__dict__.copy()
        # Custom serialization: convert `date_joined` to a string
        data["date_joined"] = self.date_joined.strftime("%Y-%m-%d %H:%M:%S")
        # Exclude email from output
        data.pop("email")
        return data

In [96]:
# Example usage
try:
    user = User(
        id=1,
        username="testuser",
        email="user@example.com",
    )
    print("User created successfully!")
    print("Serialized user:", user.model_dump_json())
except ValueError as e:
    print("Validation error:", e)

User created successfully!
Serialized user: {"id":1,"username":"testuser","date_joined":"2025-02-07 15:20:33","is_active":true}


## Strict mode and data coercion

By default, Pydantic is tolerant to common incorrect types and coerces data to the right type — e.g. a numeric string passed to an int field will be parsed as an int.

Pydantic also has as strict mode, where types are not coerced and a validation error is raised unless the input data exactly matches the expected schema.

But strict mode would be pretty useless when validating JSON data since JSON doesn't have types matching many common Python types like datetime, UUID or bytes.

To solve this, Pydantic can parse and validate JSON in one step. This allows sensible data conversion (e.g. when parsing strings into datetime objects).

In [97]:
class Meeting(BaseModel):
    when: datetime
    where: bytes

In [98]:
m = Meeting.model_validate({'when': '2020-01-01T12:00', 'where': 'home'})

m

Meeting(when=datetime.datetime(2020, 1, 1, 12, 0), where=b'home')

In [99]:
try:
    m = Meeting.model_validate(
        {'when': '2020-01-01T12:00', 'where': 'home'}, strict=True
    )
except ValidationError as e:
    print(e)

2 validation errors for Meeting
when
  Input should be a valid datetime [type=datetime_type, input_value='2020-01-01T12:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/datetime_type
where
  Input should be a valid bytes [type=bytes_type, input_value='home', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/bytes_type


In [100]:
m_json = Meeting.model_validate_json(
    '{"when": "2020-01-01T12:00", "where": "home"}'
)

m_json

Meeting(when=datetime.datetime(2020, 1, 1, 12, 0), where=b'home')

`model_validate()`

Purpose: Validate and parse data (usually untrusted or externally sourced) into a Pydantic model.

Behavior: Takes a dictionary as input and validates it according to the model’s schema.

Common Use Case: When data comes from external sources (e.g., APIs, JSON payloads, or user input) and you want to ensure it adheres to the model's rules.

In [101]:
class User(BaseModel):
    id: int
    username: str
    email: str
    password: str
    dob: datetime|None = None

In [102]:
data = {
    "id": 1,
    "username": "freethrow",
    "email": "email@gmail.com",
    "password": "somesecret",  # Suppose this came from an API payload
}
user = User.model_validate(data)
user

User(id=1, username='freethrow', email='email@gmail.com', password='somesecret', dob=None)

In [103]:
data = {
    "id": "one",
    "username": "freethrow",
    "email": "email@gmail.com",
    "password": "somesecret",
}

try:
    user = User.model_validate(data)
except Exception as e:
    print(e)

1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='one', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


`model_validate_json()` accepts a JSON string (useful when working with APIs)
* you could use this instead of `model_validate(json.loads(...))`, since it should be faster

In [104]:
import json

# Example JSON data
json_data = '''
{
    "id": 1,
    "username": "freethrow",
    "email": "email@gmail.com",
    "password": "somesecret",
    "dob": "1975-05-13T00:00:00"
}
'''

# Use model_validate_json to validate and parse the JSON string
user = User.model_validate_json(json_data)

user

User(id=1, username='freethrow', email='email@gmail.com', password='somesecret', dob=datetime.datetime(1975, 5, 13, 0, 0))

## Other ways of creating schemas

Pydantic provides four ways to create schemas and perform validation and serialization:

* BaseModel — Pydantic's own super class with many common utilities available via instance methods.
* Pydantic dataclasses — a wrapper around standard dataclasses with additional validation performed.
* TypeAdapter — a general way to adapt any type for validation and serialization. This allows types like TypedDict and NamedTuple to be validated as well as simple types (like int or timedelta) — all types supported can be used with TypeAdapter.
* validate_call — a decorator to perform validation when calling a function.

In [105]:
# Example - schema based on a TypedDict

from typing_extensions import NotRequired, TypedDict

class Meeting(TypedDict):
    when: datetime
    where: bytes
    why: NotRequired[str]

In [106]:
from pydantic import TypeAdapter

meeting_adapter = TypeAdapter(Meeting)
m = meeting_adapter.validate_python(  
    {'when': '2020-01-01T12:00', 'where': 'home'}
)

m

{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}

In [107]:
meeting_adapter.dump_python(m, exclude={'where'})  

print(meeting_adapter.json_schema())

{'properties': {'when': {'format': 'date-time', 'title': 'When', 'type': 'string'}, 'where': {'format': 'binary', 'title': 'Where', 'type': 'string'}, 'why': {'title': 'Why', 'type': 'string'}}, 'required': ['when', 'where'], 'title': 'Meeting', 'type': 'object'}


## pydantic Settings

```shell
pipenv install pydantic-settings
```

In [108]:
# from pydantic import BaseSettings # this was moved to pydantic-settings
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    api_url: str = Field(default="")
    secret_key: str = Field(default="")
    
    class Config:
        env_file = ".env"

In [109]:
print(Settings().model_dump())

{'api_url': 'https://api.com/v2', 'secret_key': 's3cretstr1n6'}


But, if you set an environment variable, it will take precedence over the .env file

In [110]:
# eg
import os
os.environ["API_URL"] = 'http://localhost:8000'

print(Settings().model_dump())

{'api_url': 'http://localhost:8000', 'secret_key': 's3cretstr1n6'}


import BaseSettings
class Settings(BaseSettings):
api_url: str = Field(default="")
secret_key: str = Field(default="")
class Config:
env_file = ".env"
print(Settings().model_dump())